# MINIMAL WORKING EXAMPLE

This notebook will execute a quick test to ensure the correctness of the installation.

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

from dlroms import*
from dlroms.roms import gramschmidt
import dlroms.fespaces as fe

import numpy as np
import matplotlib.pyplot as plt

import torch

import sys
sys.path.insert(1, '../code')

from PODlib import *
from variabilitylib import *

## Data loading

In [ ]:
# DATA
data_path = "../data/gaussian_data.npz"

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Data not found at {data_path}. Please run 'python download_data.py' in the root folder.")

dataset = np.load(data_path)
mu, u = dv.tensor(dataset['mu']), dv.tensor(dataset['u'])

ndata, nh = u.shape
p = mu.shape[-1]

# MESH
mesh_path = "../data/gaussian_mesh.xml"

if not os.path.exists(data_path):
    raise FileNotFoundError(f"Mesh not found at {mesh_path}. Please run 'python download_data.py' in the root folder.")

mesh = fe.loadmesh("../data/gaussian_mesh.xml")
Vh = fe.space(mesh, 'CG', 1)

# NUMBER OF TRAINING DATA
ntrain = int(0.75*ndata)

# SET SEED FOR MONTE CARLO ESTIMATES
seed = 42

## Ambient space setup

In [ ]:
nA = 30
V, svalues = POD(u[:ntrain], k = nA)
A = gramschmidt(V.unsqueeze(0)).squeeze(0)

uA = projectdown(A, u).squeeze(-1)

## Score computation

In [ ]:
# Initialize a weighted_POD object
w_POD = weighted_POD(A=A,
                     U=torch.t(uA[:ntrain,:]),
                     theta_full=torch.t(mu[:ntrain,:]),
                     n_basis=4,
                     omega_func=lambda theta, theta_i: omega_weights(theta, theta_i, lambda_penalty=1e-1))

# Define a scaling function
max_mu = mu.max(axis = 0).values
min_mu = mu.min(axis = 0).values

def scaling(theta):
    return theta * (max_mu-min_mu) + min_mu

# Initialize a LocalBasis object
POD_var = LocalBasis(q=p, 
                     module=w_POD, 
                     p_prime_index_list=np.arange(p), 
                     scaling=scaling) 

### Derivative-based scores

The expected results is:
```
For j=0 we have:
K_sup_j:	2.202137e+01 with std:	7.897736e+00
For j=1 we have:
K_sup_j:	2.693461e+01 with std:	1.352941e+01
For j=2 we have:
K_sup_j:	1.818119e+01 with std:	1.323490e+01
For j=3 we have:
K_sup_j:	1.569611e+01 with std:	9.782648e+00
```

Notice that as this score requires the solution of an optimization routine, it could take 1-2 minutes.

In [ ]:
K_0_mean, K_0_var = POD_var.K_j_tot(0, S=5, seed=seed)
print("For j=0 we have:")
print(f"K_sup_j:\t{K_0_mean:.6e} with std:\t{K_0_var:.6e}")

K_1_mean, K_1_var = POD_var.K_j_tot(1, S=5, seed=seed)
print("For j=1 we have:")
print(f"K_sup_j:\t{K_1_mean:.6e} with std:\t{K_1_var:.6e}")

K_2_mean, K_2_var = POD_var.K_j_tot(2, S=5, seed=seed)
print("For j=2 we have:")
print(f"K_sup_j:\t{K_2_mean:.6e} with std:\t{K_2_var:.6e}")

K_3_mean, K_3_var = POD_var.K_j_tot(3, S=5, seed=seed)
print("For j=3 we have:")
print(f"K_sup_j:\t{K_3_mean:.6e} with std:\t{K_3_var:.6e}")

### Sensitivity-based scores

The expected results is:
```
Sensitivity for j=0:	2.675419e-01
Sensitivity for j=1:	1.533968e-01
Sensitivity for j=2:	2.519202e-02
Sensitivity for j=3:	0.000000e+00
```

In [ ]:
sens = POD_var.sensitivity(m = 5, l = 5, seed=seed)
print(f"Sensitivity for j=0:\t{sens[0]:.6e}")
print(f"Sensitivity for j=1:\t{sens[1]:.6e}")
print(f"Sensitivity for j=2:\t{sens[2]:.6e}")
print(f"Sensitivity for j=3:\t{sens[3]:.6e}")

### Physically informed check

The final check takes into consideration not the exact values, but the logic of the computed sensitivity scores (the most relevant for the used dataset).

In [ ]:
if (sens[0] > sens[2] and sens[1] > sens[3]):
    print("✔ SENSITIVITY CHECK PASSED:\tThe parameters have been correctly identified.")
else:
    print("✘ SENSITIVITY CHECK WARNING:\tThe parameters have not been correctly identified.")